In [1]:
# Import necessary libraries
import pandas as pd, shutil, os
import yfinance as yf
import datetime as dt
import numpy as np
from pathlib import Path
import json

In [2]:
# Get S&P 500 tickers
def get_ticks():
    sp500_tickers = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]
    sp500_tickers = sp500_tickers.Symbol.to_list()
    sp500_tickers = [x.replace(".", "-") for x in sp500_tickers]
    
    return sp500_tickers
# sp500_tickers

In [3]:
# Start and End Dates (6 months)
def get_dates():
    end_date = dt.datetime.now().date().isoformat()
    start_date = dt.datetime.now() - dt.timedelta(weeks=26)
    start_date = start_date.date().isoformat()
    
    return start_date, end_date

In [4]:
# Getting the Forward P/E 
def ForwardPE():
    tick_list = []
    forwardPE = []
    count = 0
    for ticker in sp500_tickers:
        info = yf.Ticker(ticker).info
        info["Ticker"] = ticker
        forwardPE.append(info["forwardPE"])
        tick_list.append(info["Ticker"])
        count+=1
        print(count)
    pe_df = pd.DataFrame(data=forwardPE, index=tick_list, columns=["Forward P/E"])
    pe_df.sort_values(by="Forward P/E", ascending=False, inplace=True)
    
    return pe_df

In [5]:
# Calculating today's RSI
def calcRSI (ticker):
    data = yf.download(ticker, start=start_date,end=end_date, group_by="Ticker")
    data["Ticker"] = ticker
    data["Price_Change"] = data["Adj Close"].pct_change()
    data['Gains'] = data.loc[:,"Price_Change"].apply(lambda x: x if x > 0 else 0)
    data["Losses"] = data.loc[:,"Price_Change"].apply(lambda x: abs(x) if x < 0 else 0)
    data["Avg_Up"] = data.loc[:,"Gains"].ewm(com=13,adjust=False, min_periods=14).mean()
    data["Avg_Down"] = data.loc[:, "Losses"].ewm(com=13,adjust=False, min_periods=14).mean()
    data["RS"] = data.loc[:, "Avg_Up"]/data.loc[:,"Avg_Down"]
    data["RSI"] = data.loc[:,"RS"].apply(lambda x: 100 - (100/(x+1)))
    data = data.dropna()

    today = data.iloc[-1]

    return today

In [6]:
# Getting today's RSI for S&P 500
def Todays_RSI_Hi():
    stock_tick_list = []
    stock_rsi_list = []
    count = 0
#     join_data = pd.DataFrame()
    for stock in sp500_tickers:
        stockRSI = calcRSI(stock)
        stock_tick_list.append(stockRSI["Ticker"])
        stock_rsi_list.append(stockRSI["RSI"])
        count+=1
        print(count)
    rsi_df_hi = pd.DataFrame(data=(stock_rsi_list), index=stock_tick_list, columns=["RSI"])
    rsi_df_hi.sort_values(by=["RSI"], ascending=False,inplace=True)

    return rsi_df_hi